# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Шаги исследования

1. Подготовка данных
2. Обучение моделей
3. Тестирование
4. Вывод

## 1. Подготовка данных

In [1]:
import sys
!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 13.3 MB/s eta 0:00:01     |███████████████████████████▋    | 12.0 MB 13.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV
)

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

import spacy
from pymystem3 import Mystem

import warnings 
warnings.filterwarnings('ignore')


nltk.download('stopwords') 
m = Mystem()


RANDOM_STATE = 12345

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data = data.drop('Unnamed: 0', axis=1)
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [5]:
display(data['toxic'].value_counts())

print("Отношение класса 0 (нетоксичных комментариев) к классу 1 (токсичным комментариям):", 
      round(data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1], 2))

0    143106
1     16186
Name: toxic, dtype: int64

Отношение класса 0 (нетоксичных комментариев) к классу 1 (токсичным комментариям): 8.84


In [6]:
%%time

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


def lemmatize_text(text):
    text = text.lower()
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

data['lemm_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1) # Удалили колонку 'text' за ненадобностью

data.head()

CPU times: user 16min 53s, sys: 2.59 s, total: 16min 55s
Wall time: 17min 2s


,toxic,lemm_text
0,0,explanation why the edit make under my usernam...
1,0,d aww he match this background colour I be see...
2,0,hey man I be really not try to edit war it be ...
3,0,more I can not make any real suggestion on imp...
4,0,you sir be my hero any chance you remember wha...


In [7]:
# Разбиваем выборку на обучающую и тестовую
target = data['toxic']
features = data.drop(['toxic'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=RANDOM_STATE)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

In [8]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
features_test = count_tf_idf.transform(features_test['lemm_text'].values)
print(features_train.shape)
print(features_test.shape)
cv_counts = 3

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127433, 133938)
(31859, 133938)


In [9]:
%%time

classificator = LogisticRegression()
train_f1 = cross_val_score(classificator, 
                      features_train, 
                      target_train, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на CV', round(train_f1, 2))

F1 на CV 0.71
CPU times: user 52.9 s, sys: 1min 9s, total: 2min 2s
Wall time: 2min 2s


Значение `метрики F1` на кросс-валидации маленькое - **0.71**. Нужно сбалансировать классы.

Поменяем **веса классов**.

In [10]:
%%time

classificator = LogisticRegression(class_weight='balanced') 
train_f1_balanced = cross_val_score(classificator, 
                      features_train, 
                      target_train, 
                      cv=cv_counts, 
                      scoring='f1').mean()
print('F1 на CV со взвешиванием классов:', round(train_f1_balanced, 2))

F1 на CV со взвешиванием классов: 0.75
CPU times: user 52.8 s, sys: 1min 11s, total: 2min 3s
Wall time: 2min 4s


Со взвешиванием классов значение `метрики F1` получилось равным **0.75**. Результат улучшился.

## 2. Обучение моделей

Обучим модели:
1. **LogisticRegression**
2. **DecisionTreeClassifier**

In [11]:
%%time

classificator = LogisticRegression(class_weight='balanced')
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10]}]

clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)

print("Лучшее значение метрики F1:", round(clf.best_score_, 2))
print("Лучшие параметры:", clf.best_params_)

Лучшее значение метрики F1: 0.76
Лучшие параметры: {'C': 10, 'solver': 'newton-cg'}
CPU times: user 4min 4s, sys: 5min 49s, total: 9min 53s
Wall time: 9min 56s


In [12]:
%%time

classificator_dtc = DecisionTreeClassifier()
hyperparams = [{'max_depth':[x for x in range(30,51,2)]}]

clf_dtc = GridSearchCV(classificator_dtc, hyperparams, scoring='f1',cv=cv_counts)
clf_dtc.fit(features_train, target_train)

print("Лучшее значение метрики F1:", round(clf_dtc.best_score_, 2))
print("Лучшие параметры:", clf_dtc.best_params_)

Лучшее значение метрики F1: 0.71
Лучшие параметры: {'max_depth': 50}
CPU times: user 12min 39s, sys: 0 ns, total: 12min 39s
Wall time: 12min 53s


## 3. Тестирование

Лучше всего себя проявила модель логистической регрессии - **LogisticRegression**. Проверим ее качество на тестовой выборке.

In [13]:
predict_test = clf.best_estimator_.predict(features_test)
print('Значение F1 на тестовой выборке:', round(f1_score(target_test, predict_test), 2))

Значение F1 на тестовой выборке: 0.77


## Выводы

1. Выгрузили данные.
2. Выявили дисбаланс классов, учли его при обучении моделей.
3. Обучили **LogisticRegression** и **DecisionTreeClassifier**. Лучше всего себя проявила логистическая регрессия.
4. Проверили ее качество на тестовой выборке - `F1-мера = 0.77`, что удовлетворяет поставленным требованиям.